<a href="https://colab.research.google.com/github/RyuMinHo/GAI_project/blob/main/RAG_session_not_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio vllm transformers triton PyPDF2 Pillow sentence_transformers numpy typing faiss-gpu spacy pymupdf4llm fitz frontend tools semchunk

In [2]:
import gradio as gr
import torch
import faiss
import numpy as np
import spacy
from sentence_transformers import SentenceTransformer
import os
import time
import semchunk
import pymupdf as fitz
import pymupdf4llm
from vllm import LLM, SamplingParams
from typing import List, Tuple, Dict, Optional
from PIL import Image
import hashlib
import logging
import base64
import io
from PyPDF2 import PdfReader

Exception in thread Thread-5 (attachment_entry):
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 237, in listen
    sock, _ = endpoints_listener.accept()
  File "/usr/lib/python3.10/socket.py", line 293, in accept
    fd, addr = self._accept()
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy.py", line 52, in attachment_entry
    debugpy.listen(_dap_port)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/public_api.py", line 31, in wrapper
    return wrapped(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 143, in debug
    log.reraise

In [3]:
# 모델 초기화
def init_models():
    try:
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            print(f"CUDA device available: {torch.cuda.get_device_name(0)}")

        print("Loading LLaVA model...")
        llm = LLM(model="llava-hf/llava-v1.6-mistral-7b-hf",
                  dtype='half',
                  max_model_len=8192,
                  device="cuda" if torch.cuda.is_available() else "cpu",
                  gpu_memory_utilization=0.9)

        print("Loading SentenceTransformer...")
        embedder = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2',
                                       device="cuda" if torch.cuda.is_available() else "cpu")

        print("All models initialized successfully!")
        return llm, embedder
    except Exception as e:
        print(f"Error during model initialization: {str(e)}")
        return None, None


In [4]:
# 전역 변수로 모델 초기화
print("Starting model initialization...")
GLOBAL_LLM, GLOBAL_EMBEDDER = init_models()
if GLOBAL_LLM is None or GLOBAL_EMBEDDER is None:
    raise RuntimeError("Failed to initialize one or more required models")

SAMPLING_PARAMS = SamplingParams(temperature=0.7, max_tokens=512)
CHUNKER = semchunk.chunkerify('gpt-4', 200)

Starting model initialization...
CUDA device available: NVIDIA A100-SXM4-40GB
Loading LLaVA model...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


WARNING 12-03 12:15:33 config.py:1865] Casting torch.bfloat16 to torch.float16.
INFO 12-03 12:15:43 config.py:350] This model supports multiple tasks: {'generate', 'embedding'}. Defaulting to 'generate'.
INFO 12-03 12:15:43 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='llava-hf/llava-v1.6-mistral-7b-hf', speculative_config=None, tokenizer='llava-hf/llava-v1.6-mistral-7b-hf', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=F

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 12-03 12:15:50 model_runner.py:1077] Loading model weights took 14.0785 GB
INFO 12-03 12:15:51 worker.py:232] Memory profiling results: total_gpu_memory=39.56GiB initial_memory_usage=14.62GiB peak_torch_memory=14.94GiB memory_usage_post_profile=14.66GiB non_torch_memory=0.57GiB kv_cache_size=20.09GiB gpu_memory_utilization=0.90
INFO 12-03 12:15:52 gpu_executor.py:113] # GPU blocks: 10288, # CPU blocks: 2048
INFO 12-03 12:15:52 gpu_executor.py:117] Maximum concurrency for 8192 tokens per request: 20.09x
INFO 12-03 12:15:54 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 12-03 12:15:54 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
I

In [5]:
# RAG Pipeline 클래스
class RAGPipeline:
    def __init__(self):
        self.llm = GLOBAL_LLM
        self.sampling_params = SAMPLING_PARAMS
        self.embedder = GLOBAL_EMBEDDER
        self.chunker = CHUNKER
        self.index = faiss.IndexFlatL2(self.embedder.get_sentence_embedding_dimension())
        self.chunks = []
        self.processed_files = {}

    def get_file_hash(self, file_path: str) -> str:
        with open(file_path, "rb") as f:
            return hashlib.md5(f.read()).hexdigest()

    def indexing_pdf(self, pdf_path: List[str]):
        for pdf in pdf_path:
            try:
                file_hash = self.get_file_hash(pdf)
                if file_hash in self.processed_files:
                    logging.info(f"{pdf} has already been processed before")
                    continue

                self.processed_files[file_hash] = pdf
                logging.info(f"Processing new file: {pdf}")

                doc = fitz.open(pdf)
                markdown_text = pymupdf4llm.to_markdown(doc)
                doc.close()

                chunks = self.chunker(markdown_text)
                self.chunks.extend(chunks)
                chunks_embeddings = self.embedder.encode(chunks)
                self.index.add(chunks_embeddings)
            except Exception as e:
                logging.error(f"Error in indexing {pdf}: {e}")

        logging.info(f"Processed {len(pdf_path)} files. Total unique files: {len(self.processed_files)}")

    def process_query(self, query: str, top_k: int = 5) -> List[str]:
        query_embedding = self.embedder.encode([query])
        distances, indices = self.index.search(query_embedding, top_k)
        return [self.chunks[i] for i in indices[0]]

    def prompt_template(self, query: str, context: List[str]) -> str:
        system_message = """You are an AI assistant tasked with answering questions based on provided context. Your role is to:
1. Carefully analyze the given context
2. Provide accurate and relevant information
3. Synthesize a coherent response
4. Maintain objectivity and clarity
If the context doesn't contain sufficient information, state so clearly."""

        context_str = "\n".join([f"Context {i+1}: {ctx}" for i, ctx in enumerate(context)])

        prompt = f"""[INST] {system_message}

Relevant information:
{context_str}

User's Question: {query}

Instructions:
- Answer the query using only the information provided in the context.
- If the context doesn't contain enough information to fully answer the query, acknowledge this limitation in your response.
- Provide a concise yet comprehensive answer.
- Do not introduce information not present in the given context.
- Provide in complete sentences in English always.
- Check once again your response so that the user can be provided precise information.

Please provide your response below:
[/INST]"""

        return prompt

    def generate_response(self, query: str, context: List[str]) -> str:
        prompt = self.prompt_template(query, context)
        output = self.llm.generate([prompt], self.sampling_params)
        return output[0].outputs[0].text

    def answer_query(self, query: str, top_k: int = 5) -> str:
        retrieved_contexts = self.process_query(query, top_k)
        return self.generate_response(query, retrieved_contexts)


In [6]:
# LLaVA Image QA Processor 클래스
class LLaVAGeneralAndImageQAProcessor:
    def __init__(self):
        self.llm = GLOBAL_LLM
        self.sampling_params = SAMPLING_PARAMS

    def get_prompt(self, question: str):
        return f"""[INST]
Explain me about this image precisely in bullet points.
<image>
Your response should be in complete sentences.
\n{question} [/INST]"""

    def process_image(self, image_path, question):
        print("########image_path:", image_path)
        prompt = self.get_prompt(question)
        image = Image.open(image_path)

        inputs = {
            "prompt": prompt,
            "multi_modal_data": {"image": image},
        }

        outputs = self.llm.generate(inputs, sampling_params=self.sampling_params)
        return outputs[0].outputs[0].text


In [7]:
"""
class SessionManager:
    def __init__(self):
        self.sessions = {}

    def create_session(self, session_id):
        if session_id not in self.sessions:
            self.sessions[session_id] = {
                "rag_pipeline": RAGPipeline(),
                "img_processor": LLaVAGeneralAndImageQAProcessor(),
                "history": [],
                "mode": "General Chat",
                "mode_locked": False
            }
            return True
        return False

    def get_session(self, session_id):
        return self.sessions.get(session_id)

    def delete_session(self, session_id):
        if session_id in self.sessions:
            del self.sessions[session_id]
        return list(self.sessions.keys())
  """

'\nclass SessionManager:\n    def __init__(self):\n        self.sessions = {}\n\n    def create_session(self, session_id):\n        if session_id not in self.sessions:\n            self.sessions[session_id] = {\n                "rag_pipeline": RAGPipeline(),\n                "img_processor": LLaVAGeneralAndImageQAProcessor(),\n                "history": [],\n                "mode": "General Chat",\n                "mode_locked": False\n            }\n            return True\n        return False\n\n    def get_session(self, session_id):\n        return self.sessions.get(session_id)\n\n    def delete_session(self, session_id):\n        if session_id in self.sessions:\n            del self.sessions[session_id]\n        return list(self.sessions.keys())\n  '

In [9]:
def create_ui():
    with gr.Blocks() as demo:
        sessions = gr.State({})
        current_session = gr.State("")
        with gr.Row():
            with gr.Column(scale=1):
                new_session_btn = gr.Button("+ New Session")
                session_title_input = gr.Textbox(label="Session Title", visible=False)
                with gr.Column(elem_classes="session-container"):
                    gr.Markdown("Sessions")
                    session_list = gr.Radio(choices=[], label="")
                    delete_btn = gr.Button("🗑️ Delete Session")

            with gr.Column(scale=3):
                current_title = gr.Markdown("## No Session Selected")
                with gr.Row():
                    chat_mode = gr.Radio(choices=["General Chat", "RAG Chat"], value="General Chat", label="")
                chatbot = gr.Chatbot(height=400, render_markdown=True, show_copy_button=True, bubble_full_width=False)
                with gr.Row():
                    with gr.Column(scale=1, visible=True) as general_upload:
                        file_upload_image = gr.UploadButton("📎", file_types=[".jpg", ".jpeg", ".png"])
                    with gr.Column(scale=1, visible=False) as rag_upload:
                        file_upload_pdf = gr.File(label="PDF", file_types=[".pdf"], file_count="multiple")
                    msg = gr.Textbox(show_label=False, placeholder="메시지를 입력하세요...", container=False, scale=8)
                    send_btn = gr.Button("↑", scale=1)
                with gr.Row():
                    clear_btn = gr.Button("Clear Chat")
                with gr.Row(visible=True) as general_file_info:
                    selected_image = gr.Textbox(label="Selected Image", interactive=False)
                with gr.Row(visible=False) as rag_file_info:
                    selected_pdf = gr.Textbox(label="Selected PDF", interactive=False)

        def add_session(title, sessions):
            if not title:
                return gr.update(visible=False), gr.update(choices=list(sessions.keys()))
            if title not in sessions:
                sessions[title] = {
                    "rag_pipeline": RAGPipeline(),
                    "img_processor": LLaVAGeneralAndImageQAProcessor(),
                    "history": [],
                    "mode": "General Chat",
                    "mode_locked": False
                }
                return gr.update(visible=False), gr.update(choices=list(sessions.keys()), value=title)
            else:
                gr.Warning("이미 존재하는 세션 이름입니다.")
                return gr.update(visible=False), gr.update(choices=list(sessions.keys()))

        def switch_session(session_name, sessions):
            session = sessions.get(session_name)
            if session:
                return f"## {session_name}", session["history"], session["mode"], session_name
            return current_title, [], chat_mode.value, ""

        def clear_chat(session_name, sessions):
            session = sessions.get(session_name)
            if session:
                session["history"] = []
                return [], "", ""
            return [], "", ""

        def chat_mode_change(mode, session_name, sessions):
            session = sessions.get(session_name)
            if not session:
                return [gr.update()] * 5
            if session["mode_locked"]:
                gr.Warning("대화가 시작된 후에는 모드를 변경할 수 없습니다. 새 세션을 만들어주세요.")
                return [gr.update(value=session["mode"])] * 5
            session["mode"] = mode
            is_general = mode == "General Chat"
            return [
                gr.update(value=mode),
                gr.update(visible=is_general),
                gr.update(visible=not is_general),
                gr.update(visible=is_general),
                gr.update(visible=not is_general)
            ]

        def process_message(message, file_image, files_pdf, mode, history, session):
            try:
                if mode == "General Chat":
                    if file_image and file_image.name.lower().endswith(('.jpg', '.jpeg', '.png')):
                        #question = message if message.strip() else "이 이미지에 대해 설명해주세요."
                        return session["img_processor"].process_image(file_image, message)
                    else:
                        prompt = f"[INST] {message} [/INST]"
                        output = GLOBAL_LLM.generate([prompt], SAMPLING_PARAMS)
                        return output[0].outputs[0].text.strip()
                else:  # RAG Chat mode
                    if files_pdf:
                        pdf_paths = [f.name for f in files_pdf]
                        session["rag_pipeline"].indexing_pdf(pdf_paths)
                        return f"{len(pdf_paths)}개의 PDF가 성공적으로 처리되었습니다. 이제 문서에 대해 질문할 수 있습니다."
                    return session["rag_pipeline"].answer_query(message)
            except Exception as e:
                logging.error(f"메시지 처리 중 오류: {str(e)}")
                return f"오류가 발생했습니다: {str(e)}"

        def send_message(message, file_image, files_pdf, session_name, mode, history, sessions):
            if not message.strip() and not (file_image or files_pdf):
                return history, "", None, None, "", ""
            try:
                session = sessions.get(session_name)
                if not session:
                    return history, "", None, None, "", ""
                if not session["mode_locked"] and (message.strip() or file_image or files_pdf):
                    session["mode_locked"] = True
                    session["mode"] = mode
                current_mode = session["mode"]

                # 여기에 메시지 처리 로직 추가 (process_message 함수 호출 등)
                response = process_message(
                    message,
                    file_image if current_mode == "General Chat" else None,
                    files_pdf if current_mode == "RAG Chat" else None,
                    current_mode,
                    history,
                    session
                )
                # 예시 응답 (실제 구현에 맞게 수정 필요)


                if current_mode == "General Chat" and file_image:
                    history.append(((file_image.name, file_image), message if message.strip() else None))
                elif current_mode == "RAG Chat" and files_pdf:
                    pdf_names = [f.name for f in files_pdf]
                    history.append((f"Uploaded PDFs: {', '.join(pdf_names)}", None))
                else:
                    history.append((message, None))
                history.append((None, response))
                session["history"] = history
                return history, "", None, None, "", ""
            except Exception as e:
                logging.error(f"메시지 전송 중 오류: {str(e)}")
                return history, "", None, None, "", ""

        def delete_session(session_name, sessions, current_session):
            if session_name in sessions:
                del sessions[session_name]
            remaining_sessions = list(sessions.keys())
            if not remaining_sessions:
                gr.Warning("마지막 세션은 삭제할 수 없습니다")
                return gr.update(choices=remaining_sessions, value=session_name), current_title, chatbot, chat_mode.value, current_session
            next_session = remaining_sessions if remaining_sessions else ""
            if next_session:
                session_data = sessions[next_session]
                return gr.update(choices=remaining_sessions, value=next_session), f"## {next_session}", session_data["history"], session_data["mode"], next_session
            return gr.update(choices=remaining_sessions), "## No Session Selected", [], "General Chat", ""

        new_session_btn.click(lambda: gr.update(visible=True), outputs=session_title_input)
        session_title_input.submit(add_session, inputs=[session_title_input, sessions], outputs=[session_title_input, session_list])
        session_list.change(switch_session, inputs=[session_list, sessions], outputs=[current_title, chatbot, chat_mode, current_session])
        delete_btn.click(delete_session, inputs=[session_list, sessions, current_session], outputs=[session_list, current_title, chatbot, chat_mode, current_session])

        send_btn.click(send_message, inputs=[msg, file_upload_image, file_upload_pdf, current_session, chat_mode, chatbot, sessions],
                      outputs=[chatbot, msg, file_upload_image, file_upload_pdf, selected_image, selected_pdf])
        msg.submit(send_message, inputs=[msg, file_upload_image, file_upload_pdf, current_session, chat_mode, chatbot, sessions],
                  outputs=[chatbot, msg, file_upload_image, file_upload_pdf, selected_image, selected_pdf])
        chat_mode.change(chat_mode_change, inputs=[chat_mode, current_session, sessions],
                        outputs=[chat_mode, general_upload, rag_upload, general_file_info, rag_file_info])
        clear_btn.click(clear_chat, inputs=[current_session, sessions], outputs=[chatbot, selected_image, selected_pdf])

    return demo

# GUI 실행
demo = create_ui()
demo.launch(debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:237: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c1ef056ffe7b3f85db.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Processing /tmp/gradio/10384cd2a3565d36601778cc9bba069a194b381ccaf037191e22668a81c62bb3/1차 노동법 1.pdf...
[                                        ] (0/265)[                                        ] (  1/265)[                                        ] (  2/265)[                                        ] (  3/265)[                                        ] (  4/265)[                                        ] (  5/265)

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.51s/it, est. speed input: 476.43 toks/s, output: 69.26 toks/s]


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://c1ef056ffe7b3f85db.gradio.live
